In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import pandas as pd
df=spark.read.table("telcochurn").toPandas()
df.head()

In [2]:
df.dropna(inplace=True)
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')
df['Churn']=df['Churn'].map({'Yes':1,'No':0})

StatementMeta(, 65a050ec-035b-487a-adb6-dfee178c0273, 4, Finished, Available, Finished)

In [3]:
from sklearn.preprocessing import LabelEncoder
cat_cols=df.select_dtypes(include='object').columns
le=LabelEncoder()
for col in cat_cols:
    df[col]=le.fit_transform(df[col])

StatementMeta(, 65a050ec-035b-487a-adb6-dfee178c0273, 5, Finished, Available, Finished)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

df['TotalCharges'] = df['TotalCharges'].fillna(df['TotalCharges'].median())
X=df.drop('Churn',axis=1)
y=df['Churn']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

##print(X_train.isnull().sum())
##print(y_train.isnull().sum()
model=RandomForestClassifier()
model.fit(X_train,y_train)
y_pred=model.predict(X_test)

print(classification_report(y_test,y_pred))

StatementMeta(, 65a050ec-035b-487a-adb6-dfee178c0273, 10, Finished, Available, Finished)

              precision    recall  f1-score   support

           0       0.83      0.91      0.87      1036
           1       0.65      0.48      0.55       373

    accuracy                           0.79      1409
   macro avg       0.74      0.69      0.71      1409
weighted avg       0.78      0.79      0.78      1409



In [10]:
pred_df=X_test.copy()
pred_df['PredictedChurn']=y_pred
spark_df=spark.createDataFrame(pred_df)
spark_df.write.mode("overwrite").saveAsTable("ChurnPredictions")

StatementMeta(, 65a050ec-035b-487a-adb6-dfee178c0273, 12, Finished, Available, Finished)